In [ ]:
import os
from groq import Groq
import pandas as pd
import numpy as np

In [ ]:
apiKey = "fill_it_here"

In [ ]:
client = Groq(
    # api_key=os.environ.get("GROQ_API_KEY"),
    api_key= apiKey,
)

In [ ]:
dataset = pd.read_csv("test3label/keyphrases/done/keyphrases_AmazonMixtral_Done.csv")
# dataset = pd.read_csv("test3label/keyphrases/done/keyphrases_NetflixMixtral_Done.csv")
# dataset = pd.read_csv("test3label/keyphrases/done/keyphrases_SpotifyMixtral_Done.csv")

In [ ]:
dataset

In [ ]:
score_counts_balanced = dataset['sentiment'].value_counts().sort_index()
print(score_counts_balanced)

In [ ]:
train_data = dataset[['content', 'sentiment', 'Merged keyphrases_reduced 2']]

In [ ]:
train_data

In [ ]:
list = []
for index, row in train_data.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    
    string_message = f"""Considering the provided text and the set of topics/keyphrases I need you to analyze the sentiment and classify it into one of the five categories: 1 - negative, 2 - neutral, or 3 - positive.
                Text: "{text}"
                Topics/Keyphrases: "{topics}"
                Please just provide the sentiment classification as a single number (1, 2, or 3) and nothing more.
                """
    print("\n\n")
    print(string_message)
    print("\n")
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        # model="llama3-8b-8192",
        model="mixtral-8x7b-32768",
    )
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
len(list)

In [ ]:
rest = train_data[len(list):]
for index, row in rest.iterrows():
    text = row['content']
    topics = row['Merged keyphrases_reduced 2']
    
    string_message = f"""Considering the provided text and the set of topics/keyphrases I need you to analyze the sentiment and classify it into one of the five categories: 1 - negative, 2 - neutral, or 3 - positive.
                Text: "{text}"
                Topics/Keyphrases: "{topics}"
                Please just provide the sentiment classification as a single number (1, 2, or 3) and nothing more.
                """
    print("\n\n")
    print(string_message)
    print("\n")
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": f"""{string_message}"""
            }
    
        ],
        # model="llama3-8b-8192",
        model="mixtral-8x7b-32768",
    )
    armaz = chat_completion.choices[0].message.content
    print(f'{index}:\t{armaz}')
    list.append(armaz)

In [ ]:
list

In [ ]:
train_data['predictedSentiment'] = list

In [ ]:
import re
def findNumbers(string):
    # Find all numbers in the string
    numbers = re.findall(r'\d+', string)
    
    # Convert the numbers to integers
    numbers = [int(x) for x in numbers]
    x = None
    if(len(numbers)>0):
        x = numbers[0]-1
    else:
        x = -1
    if ((x < 0) or (x > 5)):
        x = -1
    return x

In [ ]:
formated = train_data['predictedSentiment'].apply(findNumbers)

In [ ]:
train_data['predictedSentiment'] = formated

In [ ]:
train_data

In [ ]:
train_data.to_csv('mixtralOutput/topics/filenameAmazonZeroShotMixtral-T.csv', index=False)
# train_data.to_csv('mixtralOutput/topics/filenameNetflixZeroShotMixtral-T.csv', index=False)
# train_data.to_csv('mixtralOutput/topics/filenameSpotifyZeroShotMixtral-T.csv', index=False)

In [ ]:
results = pd.read_csv("mixtralOutput/topics/filenameAmazonZeroShotMixtral-T.csv")
# results = pd.read_csv("mixtralOutput/topics/filenameNetflixZeroShotMixtral-T.csv")
# results = pd.read_csv("mixtralOutput/topics/filenameSpotifyZeroShotMixtral-T.csv")

In [ ]:
results

In [ ]:
results['predictedSentiment'].value_counts().sort_index()

In [ ]:
filtered_df = results[results["predictedSentiment"] == -1]

In [ ]:
filtered_df

In [ ]:
newResults = results[results["predictedSentiment"] != -1]

In [ ]:
newResults

In [ ]:
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def cm_analysis(y_true, y_pred, filename, labels, classes, ymap=None, figsize=(17,17)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      classes:   aliases for the labels. String array to be shown in the cm plot.
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    sns.set(font_scale=2.8)

    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.2f%%\n%d/%d' % (p, c, s)
            #elif c == 0:
            #    annot[i, j] = ''
            else:
                annot[i, j] = '%.2f%%\n%d' % (p, c)
    cm = confusion_matrix(y_true, y_pred, labels=labels, normalize='true')
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm = cm * 100
    cm.index.name = 'True Label'
    cm.columns.name = 'Predicted Label'
    fig, ax = plt.subplots(figsize=figsize)
    plt.yticks(va='center')

    sns.heatmap(cm, annot=annot, fmt='', ax=ax, xticklabels=classes, cbar=True, cbar_kws={'format':PercentFormatter()}, yticklabels=classes, cmap="Blues")
    plt.savefig(f'{filename}.png',  bbox_inches='tight')
    plt.savefig(f'{filename}.svg',  bbox_inches='tight')

In [ ]:
# Generate the confusion matrix
unique_labels = np.sort(newResults['scoreSA'].unique())
mapping = {0: 'Very Negative (1)', 1: 'Negative (2)', 2: 'Neutral (3)', 3: 'Positive (4)', 4: 'Very Positive (5)'}

# Applying mapping
classes = [mapping[value] for value in unique_labels]

In [ ]:
cm_analysis(newResults['scoreSA'], newResults['predictedSentiment'], 'API', unique_labels, classes)

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support, mean_absolute_error

accuracy = accuracy_score(newResults['scoreSA'], newResults['predictedSentiment'])
precision_ALL = precision_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
recall_ALL = recall_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
f1_ALL = f1_score(newResults['scoreSA'], newResults['predictedSentiment'], average='macro')
mae_ALL = mean_absolute_error(newResults['scoreSA'], newResults['predictedSentiment'])
precision, recall, f1, support = precision_recall_fscore_support(newResults['scoreSA'], newResults['predictedSentiment'])

In [ ]:
# Calculate accuracy for each class
for i in range(len(unique_labels)):
    temp = newResults[newResults["scoreSA"] == i]
    value = accuracy_score(temp['scoreSA'], temp['predictedSentiment'])
    print(f'Label:\t{i}\t-\tValue:\t{value}')

In [ ]:
accuracy

In [ ]:
balanced_accuracy = balanced_accuracy_score(results['scoreSA'], results['predictedSentiment'])

In [ ]:
balanced_accuracy

In [ ]:
precision_ALL

In [ ]:
recall_ALL

In [ ]:
f1_ALL

In [ ]:
mae_ALL

In [ ]:
precision

In [ ]:
recall

In [ ]:
f1

In [ ]:
support